# Data processing -- experiments

In [82]:
using Tidier

In [ ]:
using DBnomics
natl_acc = rdb("Eurostat", "nama_10_gdp", mask="B1GQ")

In [ ]:
using HTTP
using JSON
using DataFrames
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/nama_10_gdp?format=JSON&sinceTimePeriod=2000&unit=CLV10_MEUR&na_item=B1GQ&na_item=P51G&lang=en"

response = HTTP.get(url)

In [62]:
typeof(response.body)

Vector{UInt8} (alias for Array{UInt8, 1})

In [55]:
using JSONStat
import JSON
import DataStructures
jsonstatdta = JSON.parse(response,
	dicttype = DataStructures.OrderedDict)
dta = readjsondataset(jsonstatdta)

LoadError: MethodError: no method matching parse(::HTTP.Messages.Response; dicttype::UnionAll)
The function `parse` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  parse([91m::IO[39m; dicttype, inttype, allownan, null)
[0m[90m   @[39m [36mJSON[39m [90mC:\Users\MSI\.julia\packages\JSON\93Ea8\src\[39m[90m[4mParser.jl:474[24m[39m
[0m  parse([91m::AbstractString[39m; dicttype, inttype, allownan, null)
[0m[90m   @[39m [36mJSON[39m [90mC:\Users\MSI\.julia\packages\JSON\93Ea8\src\[39m[90m[4mParser.jl:443[24m[39m


In [21]:
data = JSON.parse(String(response.body)) 

Dict{String, Any} with 11 entries:
  "label"     => "Gross domestic product (GDP) and main components (output, exp…
  "status"    => Dict{String, Any}("2148"=>"p", "2018"=>"d", "548"=>"p", "1799"…
  "id"        => Any["freq", "unit", "na_item", "geo", "time"]
  "value"     => Dict{String, Any}("1766"=>28699.9, "1886"=>86739.5, "1"=>9.870…
  "version"   => "2.0"
  "updated"   => "2025-03-28T23:00:00+0100"
  "class"     => "dataset"
  "dimension" => Dict{String, Any}("geo"=>Dict{String, Any}("label"=>"Geopoliti…
  "size"      => Any[1, 1, 2, 44, 25]
  "extension" => Dict{String, Any}("status"=>Dict{String, Any}("label"=>Dict{St…
  "source"    => "ESTAT"

In [88]:
# JSONStat test

using JSONStat
import JSON
import DataStructures
oecdjsonstat = JSON.parsefile("datasets/oecd.json",
	dicttype = DataStructures.OrderedDict)
oecd = readjsondataset(oecdjsonstat)

oecd["datasetframe"]

Row,concept,area,year,value
,String,String,String,Any
1,UNR,AU,2003,5.94383
2,UNR,AU,2004,5.39663
3,UNR,AU,2005,5.04479
4,UNR,AU,2006,4.78936
5,UNR,AU,2007,4.37965
6,UNR,AU,2008,4.24909
7,UNR,AU,2009,5.59223
8,UNR,AU,2010,5.23066
9,UNR,AU,2011,5.09942


In [ ]:
using HTTP
res = HTTP.get("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true")

In [80]:
import Downloads
using CSV
natl_acc = CSV.read(Downloads.download("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true"), DataFrame)

Row,freq,unit,na_item,geo\\TIME_PERIOD\t1975 \t1976 \t1977 \t1978 \t1979 \t1980 \t1981 \t1982 \t1983 \t1984 \t1985 \t1986 \t1987 \t1988 \t1989 \t1990 \t1991 \t1992 \t1993 \t1994 \t1995 \t1996 \t1997 \t1998 \t1999 \t2000 \t2001 \t2002 \t2003 \t2004 \t2005 \t2006 \t2007 \t2008 \t2009 \t2010 \t2011 \t2012 \t2013 \t2014 \t2015 \t2016 \t2017 \t2018 \t2019 \t2020 \t2021 \t2022 \t2023 \t2024
,String1,String31,String15,String
1,A,CLV05_MEUR,B1G,AL\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t10048.4 b\t9804.4 \t10473.7 \t11012.9 \t11551.7 p\t:
2,A,CLV05_MEUR,B1G,AT\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t176594.2 \t179695.4 \t183613.0 \t190554.0 \t197178.1 \t204320.5 \t207358.1 \t210227.9 \t212983.4 \t218809.7 \t224194.7 \t232152.9 \t241143.4 \t245224.1 \t235404.3 \t239810.0 \t247618.0 \t248738.1 \t248413.9 \t250263.2 \t252957.6 \t258221.3 \t264412.6 \t271627.6 \t276482.8 \t259477.4 \t270777.5 \t287214.5 \t283822.5 \t279917.4
3,A,CLV05_MEUR,B1G,BA\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t6112.0 \t6263.5 \t6583.6 \t6848.4 \t7305.1 \t7613.8 \t8065.0 \t8559.9 \t9042.7 \t8817.8 \t8882.4 \t8974.7 \t8903.7 \t9118.0 \t9224.4 \t9351.3 b\t9650.5 \t9970.4 \t10347.6 \t10642.3 \t10350.5 \t11071.9 \t11532.7 \t11768.6 \t:
4,A,CLV05_MEUR,B1G,BE\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t220134.9 \t222152.1 \t230253.8 \t234553.8 \t242356.3 \t251047.6 \t254864.1 \t259026.0 \t261911.7 \t270219.1 \t276828.7 \t283611.6 \t294188.1 \t296506.2 \t291033.8 b\t298983.6 \t305722.8 \t306343.8 \t307054.1 \t312851.4 \t317686.5 \t320447.4 \t325019.4 \t331266.1 \t339917.3 \t325628.4 \t344150.2 \t359690.1 \t363747.1 p\t367406.1 p
5,A,CLV05_MEUR,B1G,BG\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t18104.4 \t19181.0 \t16279.8 \t16583.7 \t15103.9 \t15700.3 \t16339.7 \t17581.4 \t18311.9 \t19205.0 \t20444.8 \t21905.4 \t23339.9 \t24775.6 \t24562.9 \t25012.4 \t25487.8 \t25552.2 \t25131.1 \t25372.8 \t26119.5 \t26792.6 \t27671.7 \t28590.9 \t29573.4 \t28647.1 \t30977.8 \t32669.9 \t33082.8 \t33914.1 p
6,A,CLV05_MEUR,B1G,CH\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t270973.2 \t269132.6 \t269249.4 \t268649.6 \t271547.9 \t272766.6 \t273767.0 \t279174.0 \t287525.2 \t290918.8 \t301587.8 \t306917.1 \t307431.2 \t307261.6 \t315229.8 \t323882.1 \t336810.8 \t349722.2 \t359861.1 \t351613.1 \t362699.3 \t369754.6 \t374698.2 \t381505.9 \t390405.6 \t396747.1 \t405030.0 \t410565.5 \t422877.6 \t427969.5 \t419196.5 \t442494.5 \t455963.4 \t459192.9 \t465541.1
7,A,CLV05_MEUR,B1G,CY\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t8788.7 \t8898.4 \t9133.4 \t9691.6 \t10175.9 \t10783.0 \t11209.2 \t11626.5 \t11931.5 \t12531.2 \t13139.3 \t13758.7 \t14460.1 \t14987.4 \t14685.4 \t15026.1 \t15088.7 \t14568.5 \t13608.8 \t13367.1 \t13802.9 \t14710.3 \t15559.0 \t16552.3 \t17557.3 \t17006.4 \t18943.0 \t20338.3 \t20869.5 \t21589.0 p
8,A,CLV05_MEUR,B1G,CZ\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t76923.3 \t79854.6 \t78809.3 \t78517.4 \t79634.1 \t83044.3 \t85484.1 \t87157.8 \t89484.1 \t93751.6 \t99873.9 \t106984.5 \t112545.5 \t116476.0 \t110142.5 \t113790.8 \t115804.0 \t114852.1 \t114838.7 \t118093.1 \t123230.1 \t126367.8 \t132947.1 \t136898.4 \t141826.2 \t134662.9 \t139954.3 \t144017.9 \t144691.2 \t145367.8
9,A,CLV05_MEUR,B1G,DE\t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t: \t1736858.5 \t1772167.5 \t1751031.8 \t1790319.4 \t1822644.6 \t1837315.3 \t1875359.6 \t1917631.0 \t1954432.0 \t2015103.9 \t2054640.2 \t2055634.8 \t2044693.9 \t2076770.5 \t2095171.0 \t2177973.3 \t2250829.3 \t2272711.0 \t2131474.7 \t2229693.6 \t2313241.9 \t2324928.7 \t2335620.8 \t2387838.5 \t2422152.9 \t2479841.0 \t2549215.9 \t2579800.5 \t2

In [ ]:
using CSV
natl_acc = CSV.read("datasets/estat_nama_10_gdp.tsv.gz", DataFrame; delim="\t")

In [92]:
using CSV
natl_acc = CSV.read("datasets/estat_nama_10_gdp_en.csv.gz", DataFrame;)

Row,DATAFLOW,LAST UPDATE,freq,unit,na_item,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
,String31,String31,String1,String31,String15,String15,Int64,Float64?,String1?,String1?
1,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2019,10048.4,b,missing
2,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2020,9804.4,missing,missing
3,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2021,10473.7,missing,missing
4,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2022,11012.9,missing,missing
5,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2023,11551.7,p,missing
6,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1995,1.76594e5,missing,missing
7,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1996,1.79695e5,missing,missing
8,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1997,183613.0,missing,missing
9,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1998,190554.0,missing,missing


In [90]:
natl_acc = read_csv("datasets/estat_nama_10_gdp_en.csv.gz")

Row,DATAFLOW,LAST UPDATE,freq,unit,na_item,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
,String31,String31,String1,String31,String15,String15,Int64,Float64?,String1?,String1?
1,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2019,10048.4,b,missing
2,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2020,9804.4,missing,missing
3,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2021,10473.7,missing,missing
4,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2022,11012.9,missing,missing
5,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AL,2023,11551.7,p,missing
6,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1995,1.76594e5,missing,missing
7,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1996,1.79695e5,missing,missing
8,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1997,183613.0,missing,missing
9,ESTAT:NAMA_10_GDP(1.0),28/03/25 23:00:00,A,CLV05_MEUR,B1G,AT,1998,190554.0,missing,missing


In [ ]:
using HTTP

# dataset_id = "nama_10_gdp"
dataset_id = "nama_10_pe"
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/$(dataset_id)/?format=SDMX-CSV&compressed=true"

local_file = "datasets/$(dataset_id).csv.gz"

response = HTTP.get(url)  
open(local_file, "w") do file
    write(file, response.body)
end

## To explore

FredData -- https://juliahub.com/ui/Packages/General/FredData
SDMX -- https://juliahub.com/ui/Packages/General/SDMX
IMFData -- https://juliahub.com/ui/Packages/General/IMFData


In [27]:
using HTTP
using JSON
using DataFrames
using IterTools

function get_eurostat_from_URL(url::String)
    # Make HTTP request
    response = HTTP.get(url)
    
    if response.status == 200
        # Parse JSON response
        data = JSON.parse(String(response.body))
        
        # Extract dimensions, IDs, and sizes
        dimensions = data["dimension"]
        id_list = data["id"]  # Order of dimensions
        size = data["size"]  # Size of each dimension
        
        # Create mapping for each dimension
        mappings = Dict(dim => Dict(v => k for (k, v) in dimensions[dim]["category"]["index"]) for dim in id_list)
        
        # Generate Cartesian product of indices
        ranges = [1:s for s in size]  # Julia uses 1-based indexing
        cartesian_product = collect(product(ranges...))
        
        # Decode indices into human-readable labels
        decoded_rows = [
            Dict(dim => mappings[dim][string(index - 1)] for (dim, index) in zip(id_list, row)) 
            for row in cartesian_product
        ]
        
        # Add 'Value' column from data['value']
        for (i, row) in enumerate(decoded_rows)
            row["Value"] = get(data["value"], string(i - 1), missing)  # Default to `missing` if value is not found
        end
        
        # Convert to DataFrame
        return DataFrame(decoded_rows)
    else
        println("Failed to retrieve data: $(response.status)")
        return nothing
    end
end


get_eurostat_from_URL (generic function with 1 method)

In [29]:
df = get_eurostat_from_URL("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/namq_10_gdp?format=JSON&geo=BG&na_item=B1GQ&lang=en")

LoadError: KeyError: key "0" not found